## This Split Removes Outliers
- Does not partition on whether the patient ended up dying or not

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
notes = pd.read_csv('../../data/notes_with_interval.csv')

notes.head()

,note_id,id,note_type,note_seq,charttime,text,icu_death,interval
0,17915608-RR-64,20008098,RR,64,1975-02-06 16:23:00,EXAMINATION: CHEST (PORTABLE AP)\n\nINDICATIO...,0,3
1,17915608-RR-65,20008098,RR,65,1975-02-07 10:50:00,EXAMINATION: CHEST (PORTABLE AP)\n\nINDICATIO...,0,2
2,17915608-RR-66,20008098,RR,66,1975-02-07 20:17:00,INDICATION: ___ year old man s/p RUL lobectom...,0,2
3,17915608-RR-67,20008098,RR,67,1975-02-08 12:20:00,INDICATION: ___ year old man s/p VATS to Open...,0,1
4,17915608-RR-68,20008098,RR,68,1975-02-09 07:26:00,INDICATION: ___ year old man s/p open RUL lob...,0,1


In [3]:
dynamic = pd.read_csv('../../data/dynamic_cleaned.csv')

dynamic.head()

,id,charttime,aniongap,bicarbonate,bun,calcium,chloride,creatinine,glucose,sodium,potassium
0,26115624,9/7/50 0:22,12.0,22.0,9.0,7.9,111.0,0.6,97.0,141.0,3.6
1,21792938,4/13/28 14:18,11.0,28.0,46.0,7.1,96.0,5.0,95.0,130.0,5.2
2,28398464,12/9/34 8:10,18.0,21.0,24.0,10.3,100.0,0.7,93.0,135.0,4.0
3,28478629,10/8/96 5:30,17.0,27.0,86.0,7.4,96.0,3.3,142.0,135.0,4.7
4,22195489,9/18/45 21:05,17.0,29.0,40.0,11.3,97.0,6.4,118.0,139.0,4.2


In [4]:
static = pd.read_csv('../../data/static.csv')
static = static[static['id'].isin(dynamic['id'])]
static.head()

,id,hosp_admittime,hosp_dischtime,icu_intime,icu_outtime,los_icu,icu_death,gender,race,admission_age,...,atrial_fibrillation,malignant_cancer,chf,ckd,cld,copd,diabetes,hypertension,ihd,stroke
11,21999692,2/20/77 21:08,3/6/77 16:40,2/22/77 16:35,2/25/77 17:54,3.05,0,M,BLACK/AFRICAN AMERICAN,55.139306,...,0,0,0,0,0,1,0,0,0,0
22,25936663,1/11/45 23:02,1/22/45 16:06,1/12/45 15:10,1/13/45 18:50,1.15,0,M,WHITE,65.032170,...,0,1,0,0,1,0,0,1,1,0
25,25675339,11/2/83 21:12,11/19/83 15:36,11/6/83 20:59,11/10/83 0:44,3.16,0,M,WHITE,61.837481,...,0,0,0,0,0,0,0,0,0,0
30,27993048,11/19/67 8:23,12/25/67 14:53,11/26/67 16:26,12/5/67 16:53,9.02,0,F,WHITE,56.881238,...,0,0,1,1,0,0,1,1,1,0
32,22987108,6/10/46 16:37,7/12/46 0:00,6/22/46 11:46,7/13/46 0:27,20.53,1,M,WHITE,69.439961,...,1,0,0,1,0,0,0,1,1,0


In [5]:
static.to_csv('../../data/static_cleaned.csv', index=False)

print('saved')

saved


In [6]:
static.shape

(3146, 25)

In [7]:
Q1 = static['los_icu'].quantile(0.25)
Q3 = static['los_icu'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

print(f'lower bound: {lower_bound}')
print(f'upper bound: {upper_bound}')
print(f'sample size: {len(static)}')

lower bound: -4.38
upper bound: 12.5
sample size: 3146


In [8]:
static = static[(static['los_icu'] >= lower_bound) & (static['los_icu'] <= upper_bound)]

print(f'filtered sample size: {len(static)}')

filtered sample size: 2854


In [9]:
static_train, static_test = train_test_split(static, test_size=0.10)

static_test.to_csv('../../data/regression/no-outliers/combined/static_test.csv', index=False)

print('saved')

saved


In [10]:
static_train.shape

(2568, 25)

In [11]:
static_train, static_val = train_test_split(static_train, test_size=0.10)


static_train.to_csv('../../data/regression/no-outliers/combined/static_train.csv', index=False)
static_val.to_csv('../../data/regression/no-outliers/combined/static_val.csv', index=False)

print('saved')

saved
